# BULLKpy paper

In [ ]:
1) Best way to classify tumor types (lung: LUAD/LUSC vs NSCLC–SCLC, etc.)

Use a 2-layer scheme: TCGA project (hard truth) + clinical histology (refinement)

For TCGA, the most reproducible primary grouping is the TCGA project code (e.g., LUAD, LUSC, BRCA…). This is stable and unambiguous.

Then optionally refine using clinical columns when you need within-project splits (e.g., stage, grade, MSI, HPV status, ER/PR/HER2, etc.).

Important lung caveat (so you don’t get trapped)
	•	TCGA lung cohorts are primarily NSCLC: LUAD (adenocarcinoma) and LUSC (squamous).
	•	Classic SCLC is not a TCGA tumor type cohort in the same way as LUAD/LUSC. If you truly want NSCLC vs SCLC, you’ll likely need an external SCLC cohort (e.g., a published SCLC RNA-seq dataset) as a second repository (which you already said you might include).

Recommendation for the manuscript
	•	In TCGA-only analyses: treat lung as LUAD vs LUSC (NSCLC subtypes).
	•	In an “extension dataset” section: add SCLC and show that NE-high biology + dispersion behaves consistently.

Practical implementation
	•	Define tumor_type = adata.obs["project_id"] (or your equivalent metadata column).
	•	Add a tissue = primary_site or organ_system layer if you want pan-cancer analyses stratified by organ.

In [ ]:
2) How to measure dispersion (gene-level and pathway-level) per tumor type

You want measures that are:
	•	interpretable,
	•	robust to mean–variance effects,
	•	comparable across tumor types,
	•	useful per-sample and per-tumor-type.

2A) Per-sample dispersion (within a sample across genes)

This answers: “Is this tumor transcriptome ‘noisy/heterogeneous’ overall?”

Use normalized log expression (e.g. log1p_cpm).

Option A (simple & robust): median absolute deviation (MAD)
For each sample s, compute:
	•	D_s = \mathrm{MAD}(\{x_{gs}\}_{g \in G})
where x_{gs} is log expression for gene g in sample s, and G is a chosen gene set.

Key choice: which genes?
	•	Use highly expressed / variable genes, or
	•	Use a common gene universe across cancers (e.g., protein-coding genes minus mitochondrial/ribosomal), consistent across tumor types.

Option B (mean-variance corrected): dispersion on standardized genes
Within each tumor type t, z-score each gene across samples:
	•	z_{gs} = (x_{gs} - \mu_{g,t}) / \sigma_{g,t}
Then for each sample:
	•	D_s = \mathrm{MAD}(\{z_{gs}\}_g)

This makes dispersion comparable across tumor types and avoids “high mean → high variance” artifacts.

Reviewers like Option B because it’s clearly “heterogeneity relative to the cohort”.

⸻

2B) Gene-level dispersion per tumor type (across patients)

This answers: “Which genes are most heterogeneous in this cancer type?”

For each tumor type t and gene g:
	•	\mathrm{Disp}_{g,t} = \mathrm{MAD}(\{x_{gs}\}_{s \in t})  (or variance)

Then you can:
	•	rank genes by dispersion,
	•	compare NE markers vs others,
	•	test whether NE genes are enriched among high-dispersion genes.

Mean–variance correction (optional but strong):
Fit a mean–variance trend per tumor type and use residual dispersion:
	•	compute mean and variance per gene in tumor type,
	•	model var ~ mean (loess/spline),
	•	use residuals as “excess dispersion”.

This is extra work, but it’s very publishable if BULLKpy can make it easy.

⸻

2C) Pathway-level dispersion per tumor type

This is the most “BULLKpy-ish” and ties directly to signatures, GSEA, and leading-edge.

Approach:
	1.	Compute a pathway score per sample (NE, EMT, hypoxia, cell cycle, etc.)
	2.	For each tumor type, quantify dispersion of that score across samples

Let S_{p,s} = score of pathway/signature p in sample s.
For each tumor type t:
	•	\mathrm{Disp}_{p,t} = \mathrm{MAD}(\{S_{p,s}\}_{s \in t})

Then you get a matrix (pathway × tumor type) of dispersions → heatmap → identify “programs that are unusually heterogeneous” in certain cancers.

Why this is good
	•	It avoids gene-level noise,
	•	It’s interpretable biologically,
	•	It’s a natural on-ramp to: “NE is among the most heterogeneous programs in tumors X”.

In [ ]:
3) Look for NE among them (NE vs non-NE “two sides”)

Here’s the cleanest combined analysis:

Step 1 — Compute NE score per sample
	•	NE_score[s]

Step 2 — Define “non-NE counterpart”

Pick one (depends on your biology focus):
	•	Epithelial / luminal lineage score
	•	Immune infiltration score
	•	Cell-cycle/proliferation score
	•	EMT score

Call it nonNE_score[s].

Then define:
	•	NE–nonNE axis: axis = z(NE_score) − z(nonNE_score)
This captures the “tradeoff” and tends to be more stable than NE alone.

Step 3 — Relate dispersion to NE biology

For each tumor type t, test:
	•	correlation(NE_score, sample_dispersion)
	•	correlation(axis, sample_dispersion)
	•	compare dispersion between NE-high vs NE-low groups (top/bottom quartiles)

Step 4 — Interpret drivers
	•	DE: NE-high vs NE-low within tumor type
	•	GSEA: pathways enriched in NE-high
	•	Leading-edge overlap: identify core genes that repeatedly drive NE-related enrichment
	•	Oncoprint: show whether NE-high + high-dispersion tumors have distinctive alterations (even if only as a descriptive plot)

In [ ]:
Suggested minimal deliverables (fast, publishable)

If you want this to move quickly and cleanly:
	1.	Pan-cancer heatmap: pathway dispersion (rows) × tumor types (cols), with NE highlighted
	2.	Within selected tumor types (e.g., prostate, bladder, lung LUAD/LUSC):
	•	NE score distribution
	•	dispersion vs NE scatter + regression
	•	KM curves for (NE-high & high-dispersion) vs others
	3.	Leading-edge overlap network/heatmap for NE-associated GSEA in one or two tumor types
	4.	Oncoprint for alterations in NE-high/high-dispersion vs rest (even if exploratory)

In [2]:
adata

NameError: name 'adata' is not defined

To evaluate whether neuroendocrine (NE) programs interact with transcriptional heterogeneity to impact patient outcome, we fitted a Cox proportional hazards model including NE score (high vs low), metaprogram heterogeneity (high vs low), and their interaction, while stratifying by tumor type (TCGA Project_ID) to allow for tumor-specific baseline hazards. Both NE score and heterogeneity were associated with adverse outcome, and importantly, the interaction term was significant, indicating that the prognostic impact of NE programs is amplified in tumors with high transcriptional heterogeneity.